In [ ]:
from openai import OpenAI

client = OpenAI()


def agent_loop(user_query):
    """
    Runs an agent that uses OpenAI's Responses API with built-in web_search tool.
    
    The Responses API automatically executes tools on the backend, so we just need to:
    1. Send the query with tools enabled
    2. Parse the response to show what happened
    3. Display reasoning, actions taken, and observations
    
    Args:
        user_query: the user's question
    
    Returns:
        final answer from the agent
    """
    print("=" * 80)
    print(f"USER QUERY: {user_query}")
    print("=" * 80)
    
    # Call the Responses API with web_search tool enabled
    # The API will automatically decide if/when to use the tool
    response = client.responses.create(
        model="gpt-5-mini",
        input=[{"role": "user", "content": user_query}],
        tools=[{"type": "web_search"}]
    )
    
    print(f"\n{'─' * 80}")
    print("AGENT EXECUTION TRACE")
    print('─' * 80)
    
    # Track actions and observations
    action_count = 0
    obs_count = 0
    
    # Process each item in the response
    # The response contains reasoning items, tool calls, and output messages
    for item in response:
        item_type = item.type if hasattr(item, 'type') else type(item).__name__
        
        if item_type == 'reasoning':
            # Internal reasoning step (usually empty in web_search responses)
            print(f"\n💭 REASONING STEP")
            if hasattr(item, 'summary') and item.summary:
                for summary_item in item.summary:
                    print(f"   {summary_item}")
        
        elif item_type == 'web_search_call':
            # This is an ACTION the agent took
            action_count += 1
            print(f"\n🔧 ACTION {action_count}: Web Search")
            
            if hasattr(item, 'action'):
                action = item.action
                action_type = action.type if hasattr(action, 'type') else 'unknown'
                
                if action_type == 'open_page':
                    print(f"   Type: Open Page")
                    print(f"   URL: {action.url}")
                elif action_type == 'search':
                    print(f"   Type: Search Query")
                    if hasattr(action, 'query'):
                        print(f"   Query: {action.query}")
            
            status = item.status if hasattr(item, 'status') else 'unknown'
            print(f"   Status: {status}")
            
            # The observation is implicitly the web page content/search results
            # that the model used to generate its response
            obs_count += 1
            print(f"\n📊 OBSERVATION {obs_count}: Web search completed")
            print(f"   (Results integrated into model's knowledge)")
        
        elif item_type == 'message':
            # This is the final output from the agent
            print(f"\n✅ FINAL OUTPUT:")
            print('─' * 80)
            
            if hasattr(item, 'content'):
                for content_item in item.content:
                    if hasattr(content_item, 'text'):
                        print(content_item.text)
                        
                        # Show citations if present
                        if hasattr(content_item, 'annotations') and content_item.annotations:
                            print(f"\n📚 CITATIONS:")
                            for i, annotation in enumerate(content_item.annotations, 1):
                                if hasattr(annotation, 'url'):
                                    title = annotation.title if hasattr(annotation, 'title') else annotation.url
                                    print(f"   [{i}] {title}")
                                    print(f"       {annotation.url}")
    
    print("\n" + "=" * 80)
    print("AGENT COMPLETE")
    print("=" * 80)
    
    # Extract final text for return value
    final_text = ""
    for item in response:
        if hasattr(item, 'type') and item.type == 'message':
            if hasattr(item, 'content'):
                for content_item in item.content:
                    if hasattr(content_item, 'text'):
                        final_text = content_item.text
                        break
    
    return final_text


# Run the agent
result = agent_loop(
    "What are the latest LLM news from Simon Willison at https://simonwillison.net/ ?"
)

In [5]:
def llm_call(messages, tools):
    """
    Calls the OpenAI Responses API with the current conversation state.
    
    Args:
        messages: list of message dicts (user/assistant messages, tool calls, tool results)
        tools: list of tool definitions
    
    Returns:
        response object from the API
    """
    response = client.responses.create(
        model="gpt-5-mini",
        input=messages,
        tools=tools
    )
    return response

response = llm_call("latest article from https://simonwillison.net/", tools=[{"type": "web_search"}])
response.output

[ResponseReasoningItem(id='rs_0ae58b0aff1334a200695d02b42cf88196a43d3d8eba9c025a', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseFunctionWebSearch(id='ws_0ae58b0aff1334a200695d02b572308196addff552a54c08b5', action=ActionOpenPage(type='open_page', url='https://simonwillison.net/'), status='completed', type='web_search_call'),
 ResponseReasoningItem(id='rs_0ae58b0aff1334a200695d02b62b088196a37954b3bcfdce44', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseOutputMessage(id='msg_0ae58b0aff1334a200695d02c13b408196ab5f949d0b930c57', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=285, start_index=236, title='Simon Willison’s Weblog', type='url_citation', url='https://simonwillison.net/')], text='The most recent post on Simon Willison’s weblog is "It’s hard to justify Tahoe icons" (Jan. 5, 2026 — 7:30 pm). Simon links to and quotes Nikita Prokopov’s critique of macOS Tahoe’s new me